# Create example system to be loaded later for examples

### A 3 well test potential in 2D with ToyDynamics and a Langevin Integrator

In [1]:
import openpathsampling as paths
import numpy as np
import openpathsampling.engines.toy as toys

In [2]:
# Toy_PES supports adding/subtracting various PESs. 
# The OuterWalls PES type gives an x^6+y^6 boundary to the system.
pes = (
    toys.OuterWalls([1.0, 1.0], [0.0, 0.0]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [0.0, 0.4]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [-0.5, -0.5]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [0.5, -0.5])
)

topology=toys.Topology(
    n_spatial = 2,
    masses =[1.0, 1.0],
    pes = pes
)

template = toys.Snapshot(
    coordinates=np.array([[-0.5, -0.5]]), 
    velocities=np.array([[0.0,0.0]]),
    topology=topology
)
integ = toys.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5)

options={
    'integ' : integ,
    'n_frames_max' : 5000,
    'nsteps_per_frame' : 10
}

engine = toys.Engine(
    options=options,
    template=template
)
engine.initialized = True

In [3]:
st = paths.Storage('setup_3well2D_toysystem.nc', mode='w', template=template)

In [4]:
st.tag['engine'] = engine

In [5]:
st.close()

### Alanine Test System

In [6]:
import openpathsampling.engines.openmm as peng

import mdtraj as md
import simtk.openmm as mm
from simtk.openmm import app
from simtk import unit

In [7]:
template = peng.snapshot_from_pdb("../data/Alanine_solvated.pdb")

In [8]:
st = paths.Storage(filename="setup_alanine.nc", template=template, mode='w')

In [9]:
topology = peng.to_openmm_topology(template)            

# Generated using OpenMM Script Builder
# http://builder.openmm.org

forcefield = app.ForceField(
    'amber96.xml',  # solute FF
    'tip3p.xml'     # solvent FF
)

# OpenMM System
system = forcefield.createSystem(
    topology, 
    nonbondedMethod=app.PME, 
    nonbondedCutoff=1.0*unit.nanometers, 
    constraints=app.HBonds, 
    rigidWater=True, 
    ewaldErrorTolerance=0.0005
)

# OpenMM Integrator
integrator = mm.LangevinIntegrator(
    300*unit.kelvin, 
    1.0/unit.picoseconds, 
    2.0*unit.femtoseconds
)
integrator.setConstraintTolerance(0.00001)

# Engine options
options = {
    'nsteps_per_frame': 10,
    'platform': 'CPU'
}

engine = peng.Engine(
    template, 
    system,
    integrator,
    options
)

In [10]:
st.tag['engine'] = engine

In [11]:
psi_atoms = [6,8,14,16]
psi = paths.CV_MDTraj_Function("psi", md.compute_dihedrals, indices=[psi_atoms])

phi_atoms = [4,6,8,14]
phi = paths.CV_MDTraj_Function("phi", md.compute_dihedrals, indices=[phi_atoms])

In [12]:
st.tag['phi'] = phi
st.tag['psi'] = psi

In [13]:
st.cvs.sync()

In [14]:
st.close()